In [1]:
import pandas as pd
import numpy as np
import random
import time
import os
import re
from tqdm import tqdm 
from datetime import datetime

import spacy
from spacy.matcher import Matcher

In [2]:
nlp = spacy.load('en_core_web_sm')